In [ ]:
!pip install python-dotenv
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 設定 APIKEY
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# **Intorduction to 🤗 Hugging Face and Transformers Library**

[🤗 hugging face NLP course](https://huggingface.co/learn/nlp-course/chapter1/1)


NLP 是語言學和機器學習領域，專注於理解與人類語言相關的一切。 NLP 任務的目標不僅是單獨理解單字，而且能夠理解這些單字的上下文。</p>
NLP 常見的任務有：
   1. **Classifying whole sentences 文本分類（分類整句）**：將整個句子進行分類
      - 情感分析：「這部電影很棒！」 -> positive
      - 垃圾郵件檢測：「你的 iphone 已被嚴重損壞」-> spam
   2. **Classifying each word in a sentence 單詞分類**：對一句話中的所有字進行分類
      - 語法分析：「他跑得快」-> 他（代詞）跑（動詞）得（副詞）快（形容詞）
      - 命名實體 NER
   3. **Sentence Generation**
      1. **填充遮蔽詞**：「像這種要求，我這輩子[mask]！」-> [mask] 預測為 沒聽過
      2. **自動生成**：「今天天氣如何」-> 「今天天氣非常晴朗適合外出」
      3. **翻譯**：「你好」-> 「Hello」
      4. **摘要（問答）**
         1. 從文本中提取答案 Extractive QA：「法國的首都是巴黎。   首都在哪」-> 巴黎（藉由巴黎在原文的 index 抓出來）
         2. 以生成模型進行摘要 Generative QA：「法國的首都是巴黎。   首都在哪」-> 首都在巴黎（使用生成模型，例如 chatgpt）

## **`pipeline` in Hugging Face transformers**
``transformers`` 為 🤗Hugging face 提供的套件，讓開發者可以創建、使用 Hugging face hub 上 NLP、LLM 的模型</p>
> Hugging face hub 上的模型不只有 transformer，任何人都可以上傳任何類型的模型或資料集

在 `transformers` 中最高階的函數是 `pipeline`，
該函數將使用模型需要的預處理、推理與後處理串連起來，</p>
傳入指定的 task，`pipeline` 會自動以適合的模型進行推理（預測）。

可以從 [hub](https://huggingface.co/models) 透過 Tasks、Languages 篩選找到自己想要應用的模型</p>
從 1. [task summary](https://huggingface.co/docs/transformers/task_summary) 2. [Tasks](https://huggingface.co/tasks) 找到支援的 NLP 相關任務

### **使用 pipeline 完成常見 NLP 任務**

1. 情感分析 aka 分類問題

In [ ]:
from pprint import pprint
from transformers import pipeline
pipe = pipeline(task="sentiment-analysis")
pipe("this is awesome!!!")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9998723268508911}]

2. 命名實體</p>



在 ner 任務中，模型會對所有字詞（token） 進行分類，得到：</p>
1. 該字詞（token）對應的 entity
2. score 機率值
3. 以及對應到文本的起始結束位置

In [ ]:
ner_pipe = pipeline(task="ner",
                # model='dslim/bert-base-NER'
                )
ner_pipe("Hugging Face is a French company based in New York City.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

[{'entity': 'I-ORG',
  'score': 0.9967675,
  'index': 1,
  'word': 'Hu',
  'start': 0,
  'end': 2},
 {'entity': 'I-ORG',
  'score': 0.92930275,
  'index': 2,
  'word': '##gging',
  'start': 2,
  'end': 7},
 {'entity': 'I-ORG',
  'score': 0.9763208,
  'index': 3,
  'word': 'Face',
  'start': 8,
  'end': 12},
 {'entity': 'I-MISC',
  'score': 0.99828726,
  'index': 6,
  'word': 'French',
  'start': 18,
  'end': 24},
 {'entity': 'I-LOC',
  'score': 0.99896204,
  'index': 10,
  'word': 'New',
  'start': 42,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9986792,
  'index': 11,
  'word': 'York',
  'start': 46,
  'end': 50},
 {'entity': 'I-LOC',
  'score': 0.9992418,
  'index': 12,
  'word': 'City',
  'start': 51,
  'end': 55}]

### **練習 #1**

使用 Extractive QA model 以 `pipeline` 做 question-answering 任務：
- **給定文本**：the name of repo is bert-base-uncased
- **問題目標**：問模型 repo 的名稱
- **預期答案**：bert-base-uncased

In [ ]:
# TODO
from transformers import pipeline
ner_pipe = pipeline(task="question-answering",
                # model='dslim/bert-base-NER'
                )
ans = ner_pipe(question="問模型 repo 的名稱",
         context="the name of repo is bert-base-uncased")
# practice 1 不需要特別指定模型，pipeline 預設載入 distilbert-base-cased-distilled-squad,
# 其為 Extractive QA 類摘要模型

print(ans)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'score': 0.7133434414863586, 'start': 20, 'end': 37, 'answer': 'bert-base-uncased'}


### **利用 Conversation class 與 text-generation model 實作 chatbot**

In [ ]:
from pprint import pprint

from torch import cuda, bfloat16
from transformers import pipeline
from transformers import BitsAndBytesConfig, AutoConfig, AutoModelForCausalLM, AutoTokenizer
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

cuda:0


因為載入模型較大，使用 T4 GPU 時建議進行量化，以下程式為量化處理過程，</p>
在此先不贅述，有興趣的可以參考 Hugging face 官方文件～</p>

與前面範例不同的是，模型載入方法，我們透過 `AuToModelForCausalLM` 實例化模型，將其作為參數傳入 `pipeline`。

In [ ]:
model_id = 'MediaTek-Research/Breeze-7B-32k-Instruct-v1_0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = AutoConfig.from_pretrained(
    model_id
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

hf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/512M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
chatbot = pipeline(
    "text-generation",
    model=hf_model,
    tokenizer=tokenizer, # Tokenizer，要與模型匹配，主要提供 chat 模式時的特殊符號
    max_new_tokens=1024, # 模型最多可以生成多少字
    return_full_text=False # 控制 pipeline 只輸出 AI Message
)

聊天式模型，例如 ChatGPT 其實基本上是透過 text-generation 作為基礎模型，進一步訓練模型能過聊天。
所以模型的選擇，我們可以在 Huggingface 上找到 text-generation 任務的模型，應該都可以支援。</p>

比較特別的是，要做聊天任務時，模型需要一些特殊符號來區別每一段訊息是來自於 User 或是 AI 還是 System Prompt</p>
而各個模型的特殊符號不盡相同，需要去查閱官方文件。例如 Demo 使用的聯發科 Breeze 模型是透過 `[INST]` 、 `[/INST]` 以及 `<s>` 作為區隔。
所以我們在使用模型時，就會需要將文字加上這些特殊符號才能夠發揮模型聊天的能力。</p>

通常我們會使用 list of dict 的方式處存聊天的記錄，使用 role 區別 user 與 ai，content 代表內容，而 Hugging face 的模型也支援這樣的格式，例如：


```
[
    {"role": "user", "content": "嗨你好嗎"},
    {"role": "assistant", "content": "嗨您好，我是您的 AI 助理，很高興為您服務。"},
    {"role": "user", "content": "掰掰"},
    {"role": "assistant", "content": "掰掰，期待再相見"},

]

```

我們可以透過實例化 Conversation 這個 class，透過 `add_user_input` 與 `append_response` 新增歷史使用者輸入與模型回覆，將資料變為上述的資料格式再送給模型進行推理。


In [ ]:
from transformers import Conversation
conversation = Conversation() # 建立一個對話 Conversation 物件

利用 `add_user_input` 新增 user 聊天記錄

In [ ]:
conversation.add_user_input("provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?")
print(f"目前聊天記錄：{conversation.messages}") # conversation.messages 可以直接丟給 chatbot 得到回覆

目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}]


In [ ]:
# 將 conversation.messages 丟給 chatbot
chatbot_result = chatbot(conversation.messages)
print(chatbot_result)

[{'generated_text': '根據提供的信息，repo的名称是"bert-base-uncased"。'}]


將 chatbot 的回覆以 `append_respons` 的方法加入 conversation 中

In [ ]:
conversation.append_response(chatbot_result[0]['generated_text'])
print(f"目前聊天記錄：{conversation.messages}")

目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}, {'role': 'assistant', 'content': '根據提供的信息，repo的名称是"bert-base-uncased"。'}]


In [ ]:
conversation.add_user_input("那什麼是 bert?")

print(f"目前聊天記錄：{conversation.messages}")

目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}, {'role': 'assistant', 'content': '根據提供的信息，repo的名称是"bert-base-uncased"。'}, {'role': 'user', 'content': '那什麼是 bert?'}]


In [ ]:
chatbot_result = chatbot(conversation.messages)
print(f"LLM 回覆：{chatbot_result}")

print("-"*10)
conversation.append_response(chatbot_result[0]['generated_text'])
print(f"目前聊天記錄：{conversation.messages}")

LLM 回覆：[{'generated_text': ' Bert是Bidirectional Embedding Representations from Transformers的缩写，是Google自然语言处理团队所研发的一个预训练模型。它基于Transformers架构，通过同时考虑文本的前向和后向信息，实现了有针对性地语义表达和知识表达。Bert模型在各种自然语言处理任务，如情感分析、命名实体识别、问答系统等任务上表现突出，已成为当今预训练模型中的一个标准基线。'}]
----------
目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}, {'role': 'assistant', 'content': '根據提供的信息，repo的名称是"bert-base-uncased"。'}, {'role': 'user', 'content': '那什麼是 bert?'}, {'role': 'assistant', 'content': ' Bert是Bidirectional Embedding Representations from Transformers的缩写，是Google自然语言处理团队所研发的一个预训练模型。它基于Transformers架构，通过同时考虑文本的前向和后向信息，实现了有针对性地语义表达和知识表达。Bert模型在各种自然语言处理任务，如情感分析、命名实体识别、问答系统等任务上表现突出，已成为当今预训练模型中的一个标准基线。'}]


### **embedding model (feature extraction)**
[參考](https://huggingface.co/tasks/feature-extraction)


Embedding 是將文字轉換成向量的技術，使得文字可以在數學空間中表示。</p>這些向量捕捉了文字之間的語義關係，使得相似的文字在向量空間中更接近。常見的嵌入模型包括 Word2Vec、GloVe 和 BERT 等。

在 Retrieval-Augmented Generation (RAG) 中，我們會用 Embedding model 用來將查詢（query）和候選文檔（document）轉換成向量。</p>
通過計算這些向量的相似度，可以找出與查詢最相關的文檔。這些相關文檔隨後用來生成回答，增強生成模型的準確性和上下文相關性。

我們使用 sentence transformers 這個套件，可以從 [官方文檔](https://sbert.net/docs/pretrained_models.html) 尋找自己希望使用的 model，也可以在 hugging face 平台上搜尋支援 feature-extraction 的模型。</p>

另外 hugging face 也提供 [embedding model](https://huggingface.co/spaces/mteb/leaderboard) 的排行榜給大家參考

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
# 利用 encode 得到 sentence 的 embedding
embedding_model.encode("哈囉，這是一個句子")

array([ 0.03142083, -0.01894771, -0.00766944, ..., -0.02039895,
       -0.01210877,  0.03742645], dtype=float32)

In [ ]:
query = "為什麼 ML 需要做正規化"

source_sentence = [
    'Regularization is important!',
    'Dropout is important!',
    'Missing Data Handling is important!'
]

當我們有每個句子的 embedding 後就可以透過 cosine similarity 計算每個文本的相似度。

In [ ]:
import numpy as np
def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
most_related_sentence = None
max_similarity = 0

for sentence in source_sentence:
    sim = calculate_cosine_similarity(
    embedding_model.encode(query),
    embedding_model.encode(sentence)
    )

    if sim > max_similarity:
        most_related_sentence = sentence
        max_similarity = sim

    print(f"{query} vs {sentence} similarity: {sim}")

print("="*10)
print(f"與「{query}」最相似文本：{most_related_sentence}")

為什麼 ML 需要做正規化 vs Regularization is important! similarity: 0.8665975332260132
為什麼 ML 需要做正規化 vs Dropout is important! similarity: 0.805509626865387
為什麼 ML 需要做正規化 vs Missing Data Handling is important! similarity: 0.814452588558197
與「為什麼 ML 需要做正規化」最相似文本：Regularization is important!


**作業 demo**

利用 Hugging Face 的 text-generation model 與 Sentence Transformers embedding model 實作 QA 檢索聊天機器人。</p>
基於提供的資料集，使用 Embedding Cosine Similarity 檢索參考資料，再透過 LLM 生成答案。</p>

1. Baseline
   - 將 demo 中的資料，替換成我們提供 or 自己的資料集
   - 能夠檢索相似資料
   - 基於檢索的資料進行回答
2. Advanced（Optional）
   - Embedding 怎麼儲存？每次都要重新計算嗎？
   - 該如何處理太久以前的歷史資料？
   - 利用 Gradio or Hugging Face Spaces 部署、分享 Chatbot

In [ ]:
qa_data = [
    """1. 入營注意事項：役男入營前無須先行理髮，待入營後統一安排理髮，再向役男收取費用
    2. 役男可攜帶手機(不得為大陸廠牌，且不提供充電)，部隊會集中保管，於每日夜間以定時定點方式使用。
    3. 營區不提供充電，建議攜帶拋棄式(手動)刮鬍刀；不可攜帶噴霧式液體。
    4. 役男可配戴隱形眼鏡，但因基礎訓練期間生活緊湊，而隱形眼鏡需相當時間消毒清洗，建議配戴鏡片眼鏡為宜，並可多備一副眼鏡，以供替換。
    """,

    """報到入營時應該文件：
    1. 徵集令。
    2. 役男本人之國民身分證正本。
    3. 私章。
    4. 健保ＩＣ卡。
    5. 郵局存摺正面影本。
    6. 替代役役男輔導需求調查表。
    7. 個人特殊醫療用品。
    """,

    """
    替代役訓練天數
    基礎訓練：21天(含撥交日)。(自253梯次起修正)
    專業訓練：以各分發需用機關不同而有個不同期間的專業訓練。
    """,

    """
    折抵役期規定
    請備妥相關證明文件，如高中（職）以上各級學校經軍訓主管驗證加註折抵役期日數之成績單、大專集訓結訓證書、
    驗退（停役）證明書或軍事學校退學（開除）證明書等正本（驗證後退還）。82年次以前出生者，
    合計不得逾30日，83-93年次以後出生者，合計不得逾15日。已受軍事入伍訓練者，請於收到徵集令時，向戶籍地區公所提出免受基礎訓練申請。
    """,

    """
    要當兵時，健保要辦理轉出轉入嗎?
    即將入營服常備兵役之役男需持徵集令向全民健保加保單位辦理轉出，轉出日期填報「入營當月份」，轉入單位免填，後續將由國軍單位接續辦理後續轉入程序。
    """,

    """當兵期間義務役薪水的入帳戶一定要郵局嗎?
    依現行規定目前有郵局、台新、土地銀行跟合作金庫這四家金融機構可使用。"""
]

In [ ]:
from typing import List
import numpy as np

def get_answer(query: str, source: List[str]):
    most_related_sentence = None
    max_similarity = 0

    for sentence in source:
        sim = calculate_cosine_similarity(
        embedding_model.encode(query),
        embedding_model.encode(sentence)
        )

        if sim > max_similarity:
            most_related_sentence = sentence
            max_similarity = sim

    return most_related_sentence

def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
user_query = input(">>>")
conversation = Conversation()

while user_query.lower() != "bye":
    print(f"user: {user_query}")
    # 尋找最相似的文件
    answer = get_answer(user_query, qa_data)
    llm_input = f"""請你基於以下資訊回答使用者的問題
    {answer}
    ===
    問題：{user_query}
    """
    conversation.add_user_input(llm_input)
    # 將 conversation.messages 丟給 chatbot
    chatbot_result = chatbot(conversation.messages)[0]['generated_text']
    print(f"AI: {chatbot_result}")
    conversation.append_response(chatbot_result)

    user_query = input(">>>")


>>>薪水可以匯到哪些銀行
user: 薪水可以匯到哪些銀行
AI: 根據目前的規定，當兵期間義務役的水利可以在郵局、台新銀行、土地銀行，以及合作金庫等四家金融機構的帳戶入款。
>>>
user: 
AI: 報到入營時需要準備哪些文件？


KeyboardInterrupt: Interrupted by user

## My Homework

#### Baseline

In [ ]:
qa_data = [
    """野原廣志：
       廣志是個身上有 32 年房貸跟一雙臭腳、有點好色，卻又怕老婆的 35 歲上班族，但其實已經有文章分析過，在這個人設的背後，廣志其實是個不折不扣的「人生勝利組」。
       廣志畢業於名校早稻田大學，35 歲就已經可以在東京都市圈的埼玉縣擁有一幢兩層還帶院子的房子。同時，這個年紀就能在公司中擔任主管職務，年薪在稅前大約有 600-650 萬日幣，在同年齡的人之中屬於水準以上，甚至還可以從自己微薄的零用錢中借一點給部下川口。
       在生活中，廣志有時候喜歡偷看其他美女，也會因為工作應酬跟同事或客戶到夜店喝酒，但卻沒有真的做出什麼出格行為。雖然自己的零用錢到月底常常不夠，對妻子美冴卻頗為大方，才會常常出現美冴不小心手滑買了高級衣服、保養品，或一大堆只用過一次的減肥器材的橋段。
       在家庭裡，廣志作為父親的角色，在臼井儀人的筆下也從未失職過。雖然星期天經常喜歡賴床，但仍有很多野原一家在假日出遊的畫面。可以說，在廣志看似不太正經的人設背後，其實是個在主流社會定義中的好丈夫、好父親，也是個會照顧下屬的好主管。
    """,

    """野原美冴：
       美冴是個 29 歲的家庭主婦，雖然在種種的生活細節裡老是被小新的行為惹怒，例如早上起不來搭娃娃車、不小心打翻高級洗髮精、弄髒美冴新買的衣服、挑食不吃青椒、不整理玩具等等，這些族繁不及備載的事情，經常讓美冴忍不住對小新使出「憤怒鐵拳」。但其實這對母子的關係是相當緊密且融洽的。
       在對小新的教育上，美冴雖然也會受到媽媽圈中「比較孩子」的壓力，例如上文提到的「媽媽們的聚會」，小新的表現讓他在所謂「乖小孩積分評量表」裡一分都拿不到；而小新雖然也不時會被送去兒童補習班試聽（雖然部分原因是想貪點「免費試聽」的小便宜）、也曾請過家教老師，但最後卻都作罷。
       從這些劇情就不難發現，美冴即使面對這些壓力，最終還是決定讓小新順著自己的性格發展。在一股「不能讓孩子輸在起跑點」的風氣中，想要堅持這種看似簡單的決定，其實需要無比的勇氣。
       美冴與小新的另一個互動模式，也側面反映了社會對女性外貌不合理的要求。美冴的身高 159.2 公分、體重 52 公斤，BMI 其實是相當標準的 20.5 左右，即便體脂肪 29%，也還是落在正常範圍內。
       但在小新口中，母親美冴卻時常被稱為「妖怪大屁屁肚子三層肉老太婆」，還不時需要買各種減肥食品或健身器材（雖然通常只有三分鐘熱度）來幫助維持身材——這種刻薄的批評，借小新之口說出來，難道不失為一種對社會標準的諷刺嗎？
    """,

    """野原新之助：
       最後，回到主角小新。前面寫了一連串小新讓人惱怒的行為，但他真的是個如此不堪的小孩嗎？當然不是，作者其實只是用了比較誇張的表現方式，以及一些看似庸俗的行為，反面諷刺了社會為人們帶上的面具有多麼虛假。
       若沒有遇到小新，風間會是個偶包很重的小紳士、妮妮會是個壓抑自己的小淑女、松坂老師會是個看起來光鮮亮麗的上流社會女子；但在小新面前，這一切的虛假都被戳破，風間、妮妮會失態，松坂老師打腫臉充胖子的行為會露出馬腳。
       另一方面，阿呆、正男、吉永老師等原本就沒有太多偽裝的角色，則鮮少因為小新而有截然不同的行為模式。
    """
]

In [ ]:
from typing import List
import numpy as np

def get_answer(query: str, source: List[str]):
    most_related_sentence = None
    max_similarity = 0

    for sentence in source:
        sim = calculate_cosine_similarity(
        embedding_model.encode(query),
        embedding_model.encode(sentence)
        )

        if sim > max_similarity:
            most_related_sentence = sentence
            max_similarity = sim

    return most_related_sentence

def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:

user_query = input(">>>")
conversation = Conversation()

while user_query.lower() != "bye":
    print(f"user: {user_query}")
    # 尋找最相似的文件
    answer = get_answer(user_query, qa_data)
    llm_input = f"""請你基於以下資訊回答使用者的問題
    {answer}
    ===
    問題：{user_query}
    """
    conversation.add_user_input(llm_input)
    # 將 conversation.messages 丟給 chatbot
    chatbot_result = chatbot(conversation.messages)[0]['generated_text']
    print(f"AI: {chatbot_result}")
    conversation.append_response(chatbot_result)

    user_query = input(">>>")


>>>美冴會對小新使出什麼
user: 美冴會對小新使出什麼
AI: 美冴會對小新使出「憤怒鐵拳」。
>>>野原廣志還有幾年房貸
user: 野原廣志還有幾年房貸
AI: 野原廣志還有 32 年房貸。
>>>bye


#### Advanced（Optional）

1. Embedding 怎麼儲存？每次都要重新計算嗎？
2. 該如何處理太久以前的歷史資料？
3. 利用 Gradio or Hugging Face Spaces 部署、分享 Chatbot

In [ ]:
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer
import numpy as np

# 加載模型
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 假設你有一組句子
sentences = qa_data

# 生成嵌入向量
embeddings = embedding_model.encode(sentences)
print(embeddings.shape)  # 應該是 (num_sentences, embedding_dimension)

# 向量的維度
d = embeddings.shape[1]

# 建立Annoy索引
index = AnnoyIndex(d, 'euclidean')
for i in range(embeddings.shape[0]):
    index.add_item(i, embeddings[i])

index.build(10)  # 建立10棵樹

# 檢索示例
k = 2
query_embedding = embedding_model.encode(["憤怒鐵拳"])[0]
I = index.get_nns_by_vector(query_embedding, k, include_distances=True)
print("最近鄰居的索引:", I[0])
print("最近鄰居的距離:", I[1])

# 保存和加載索引
index.save('vector_index.ann')
index = AnnoyIndex(d, 'euclidean')
index.load('vector_index.ann')

(3, 384)
最近鄰居的索引: [2, 0]
最近鄰居的距離: [7.1659722328186035, 7.376850605010986]


True